## I. 패키지 밎 db- graphing 에서 테이블

In [ ]:
!pip install pymysql
!pip install sqlalchemy
!pip install mysqlclient

     |████████████████████████████████| 43 kB 296 kB/s 
     |████████████████████████████████| 87 kB 3.0 MB/s 
  Created wheel for mysqlclient: filename=mysqlclient-2.1.0-cp37-cp37m-linux_x86_64.whl size=99966 sha256=a9c8492c226e77a53ddc39d35199fce4404a121eceb2bb0658c168e7347dff4f
  Stored in directory: /root/.cache/pip/wheels/97/d4/df/08cd6e1fa4a8691b268ab254bd0fa589827ab5b65638c010b4
Successfully built mysqlclient


In [ ]:
!pip install pyecharts
!pip install chart_studio

     |████████████████████████████████| 135 kB 5.2 MB/s 
     |████████████████████████████████| 130 kB 29.3 MB/s 
     |████████████████████████████████| 64 kB 1.6 MB/s 


In [ ]:
import warnings
warnings.filterwarnings('ignore')
import time
import datetime
import pymysql
from sqlalchemy import create_engine
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import font_manager, rc

import itertools
import operator

import base64
from io import BytesIO

from datetime import datetime, timedelta

from pyecharts.charts import Bar
from pyecharts.charts import Line
import plotly.offline as plyo
import cufflinks
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
crypto_name_p = ['XRP']
for i in crypto_name_p:
  db = pymysql.connect(host='35.76.154.105', port=8999, user='root', passwd='multi1203',db='graphing', charset='utf8')
  cursor = db.cursor(pymysql.cursors.DictCursor)
  sql = f'SELECT * FROM {i}_only'
  cursor.execute(sql)
  result = cursor.fetchall()
  globals()[f'{i}'] = pd.DataFrame(result)

In [ ]:
XRP

,index,date,close_KRW,volume,positive_E,neutrual_E,negative_E,positive_K,neutrual_K,negative_K,positive_ratio_E,positive_ratio_K
0,0,2017-09-25,203.0,2.130000e-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2017-09-26,211.0,5.160000e-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2017-09-27,232.0,5.040000e-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,2017-09-28,222.0,4.980000e-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,2017-09-29,220.0,5.230000e-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1521,1521,2021-11-27,1215.0,1.644314e+08,3142.0,14.0,6397.0,13.0,6.0,20.0,0.328902,0.333333
1522,1522,2021-11-28,1220.0,2.488932e+08,NaN,NaN,NaN,10.0,8.0,17.0,NaN,0.285714
1523,1523,2021-11-29,1230.0,6.381037e+06,NaN,NaN,NaN,8.0,3.0,10.0,NaN,0.380952
1524,1524,2021-11-30,1255.0,7.274336e+06,908.0,1.0,1934.0,4.0,3.0,2.0,0.319381,0.444444


## III.테스트해보기

In [ ]:
io = pd.read_csv('io.csv')
ix = pd.read_csv('ix.csv')

In [ ]:
io['0'] = pd.to_datetime(io['0'])
ix['0'] = pd.to_datetime(ix['0'])

In [ ]:
XRP = XRP[-371:-1]
XRP = XRP.loc[::-1]
XRP.reset_index(drop=True, inplace=True)

XRP['P/(P+N)'] = (XRP['positive_K'])/((XRP['negative_K'])+(XRP['positive_K']))
XRP['D(1)P/(P+N)(t)'] = -1*(XRP['P/(P+N)'].diff().shift(-1))/1
XRP['D(2)P/(P+N)(t)'] = -1*(XRP['D(1)P/(P+N)(t)'].diff().shift(-1))/1

XRP['KR'] = 0.9 *(XRP['P/(P+N)'])+ 0.09 *(XRP['D(1)P/(P+N)(t)'])+ 0.01 *(XRP['D(2)P/(P+N)(t)'])

In [ ]:
crypto = XRP
P_date = 0
crypto = crypto.replace(0, '')

fig = go.Figure()

fig.add_trace(go.Scatter(x=crypto['date'][-(P_date):],y=crypto['close_KRW'][-(P_date):]
                         ,name='KRW',text=crypto['close_KRW'][-(P_date):],yaxis='y'))
fig.add_trace(go.Scatter(x=crypto['date'][-(P_date):],y=crypto['positive_ratio_E'][-(P_date):],
                         name='EN',text =round(crypto['positive_ratio_E'][-(P_date):],4),yaxis='y2'))
fig.add_trace(go.Scatter(x=crypto['date'][-(P_date):],y=crypto['KR'][-(P_date):],
                         name='KR',text =round(crypto['KR'][-(P_date):],4),yaxis='y3'))

fig.update_traces(
    hoverinfo="name+x+text",
    line={"width": 1},
    marker={"size": 1},
    mode="lines+markers",
    showlegend=False
)


fig.update_layout(
    xaxis=dict(
        autorange=True,
        range=["2021-10-31", "2021-11-25"],
        rangeslider=dict(
            autorange=True,
            range=["2021-10-31", "2021-11-25"]
        ),
        type="date"
    ),
    yaxis=dict(
        anchor="x",
        autorange=True,
        domain=[0.35, 1],
        linecolor="#673ab7",
        mirror=True,
        range=[-60.0858369099, 1000],
        showline=True,
        side="left",
        tickfont={"color": "#673ab7"},
        tickmode="auto",
        ticks="",
        title="KRW",
        titlefont={"color": "#673ab7"},
        type="linear",
        zeroline=False

    ),
    yaxis2=dict(
        anchor="x",
        autorange=True,
        domain=[0, 0.15],
        linecolor="#E91E63",
        mirror=True,
        range=[29.3787777032, 100000],
        showline=True,
        side="right",
        tickfont={"color": "#E91E63"},
        tickmode="auto",
        ticks="",
        title="EN",
        titlefont={"color": "#E91E63"},
        type="linear",
        zeroline=False
    ),
    yaxis3=dict(
        anchor="x",
        autorange=True,
        domain=[0.16, 0.31],
        linecolor="#E91E63",
        mirror=True,
        range=[29.3787777032, 100000],
        showline=True,
        side="right",
        tickfont={"color": "#E91E63"},
        tickmode="auto",
        ticks="",
        title="KR",
        titlefont={"color": "#E91E63"},
        type="linear",
        zeroline=False
    )
)

fig.update_layout(
    dragmode="zoom",
    hovermode="x",
    legend=dict(traceorder="reversed"),
    height=600,
    template="simple_white",
    margin=dict(
        t=100,
        b=100
    ),
)

# annotation_arrows = [dict(x=date, y=val, showarrow=True, arrowwidth=2,
#                           arrowhead=1, arrowcolor='blue', yshift=10, arrowside='end')
#                           for date, val in zip(io['0'].tolist(), io['3'].tolist())]
# fig.update_layout(annotations = annotation_arrows)

# fig.add_traces(go.Scatter(x=io['0'].tolist(), y=(io['3']*1.2).tolist(), mode = 'markers',
#                marker = dict(symbol = 'triangle-down', color = 'red')))

shape_list = []
for num in range(0, len(io)):
    shape_list.append(dict(
        type = 'rect',
        xref = 'x',
        yref = 'paper',
        x0 = io['0'][num],
        y0 = 0,
        x1 = io['0'][num]+ timedelta(days=1),
        y1 = 1,
        fillcolor = 'Red',
        opacity=0.3,
        layer = 'below',
        line_width = 0
    ))
for num in range(0, len(ix)):
    shape_list.append(dict(
        type = 'rect',
        xref = 'x',
        yref = 'paper',
        x0 = ix['0'][num],
        y0 = 0.35,
        x1 = ix['0'][num]+ timedelta(days=1),
        y1 = 1,
        fillcolor = 'black',
        opacity=0.3,
        layer = 'below',
        line_width = 0
    ))
fig.update_layout(shapes = shape_list)

fig.add_shape(go.layout.Shape(type='line', yref='y3', x0=crypto['date'][0], x1=crypto['date'][369], y0=0.5, y1=0.5,
                          line=dict(color='black', width=1)))
fig.show()